In [1]:
from biome.text import Pipeline

In [2]:
pipeline = Pipeline.load('tweet-classification/learn/preprocessedDroppedNs-lstm-no-trainable-final/model.tar.gz')

In [25]:
THRESHOLD = 0.7

## Function for getting predictions from the model, above a certain threshold

In [26]:
def get_prediction(text):
    result = pipeline.predict(text)
    return result['max_class'] if result['max_class_prob'] >= THRESHOLD else 'Other'

In [27]:
get_prediction('me gustan las hamburguesas')

'Non-healthy'

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('datasets/Twitter_cases_new.csv') ; df.head()

,Unnamed: 0,Screen Name,Anonymized,Tweet Text,preprocessed_tweet
0,0,@UnavailingSpan,Uno,Mi cobija.... Mi cobija...,mi cobija . <repeated> mi cobija . <repeated>
1,1,@UnavailingSpan,Uno,lo único que ha sobrevivido años de trasteo y ...,lo único que ha sobrevivido años de trasteo y ...
2,2,@UnavailingSpan,Uno,Todos mis ex son como kaji,todos mis ex son como kaji
3,3,@UnavailingSpan,Uno,Por eso las personas con lunares en los labios...,por eso las personas con lunares en los labios...
4,4,@UnavailingSpan,Uno,Why i am crying?,why i am crying ?


## Same idea as with the preprocessing function

In [17]:
df['prediction'] = df['preprocessed_tweet'].apply(lambda x: get_prediction(x)) # pandas df apply

In [31]:
df.head()

,Unnamed: 0,Screen Name,Anonymized,Tweet Text,preprocessed_tweet,prediction
0,0,@UnavailingSpan,Uno,Mi cobija.... Mi cobija...,mi cobija . <repeated> mi cobija . <repeated>,Positive
1,1,@UnavailingSpan,Uno,lo único que ha sobrevivido años de trasteo y ...,lo único que ha sobrevivido años de trasteo y ...,Negative
2,2,@UnavailingSpan,Uno,Todos mis ex son como kaji,todos mis ex son como kaji,Other
3,3,@UnavailingSpan,Uno,Por eso las personas con lunares en los labios...,por eso las personas con lunares en los labios...,Other
4,4,@UnavailingSpan,Uno,Why i am crying?,why i am crying ?,Other


In [32]:
df.prediction.value_counts()

Other              579
Positive           197
Negative           106
Healthy             23
Practice_sports     18
Non-healthy         10
Name: prediction, dtype: int64

In [43]:
grouped= df[['Anonymized','Tweet Text','prediction']].groupby(['Anonymized','prediction'],as_index=False).count()

grouped_pivot = grouped.pivot(index='Anonymized',columns='prediction').fillna(0)
grouped_pivot

Tweet Text                                                    
prediction    Healthy Negative Non-healthy Other Positive Practice_sports
Anonymized                                                               
Cinco             2.0      2.0         0.0  79.0     12.0             0.0
Cuatro            2.0     11.0         1.0  65.0     13.0             1.0
Diez              1.0     10.0         0.0  35.0     17.0             2.0
Doce              1.0      7.0         0.0  49.0     24.0             0.0
Dos               4.0      8.0         3.0  39.0     27.0            10.0
Nueve             1.0     10.0         2.0  24.0     14.0             2.0
Ocho              0.0     11.0         1.0  65.0      9.0             0.0
Once              0.0     10.0         0.0  32.0      6.0             1.0
Seis              7.0     12.0         2.0  53.0     16.0             0.0
Siete             1.0      5.0         1.0  37.0     11.0             2.0
Tres              2.0     12.0         0.0  43.0     29.0             0.0
Uno               2.0      8.0         0.0  58.0     19.0             0.0

# 2. Complete dataset

In [112]:
dff = pd.read_csv('datasets/Twitter_1-7_colombia_final.csv') ; dft.head()

,Unnamed: 0,Full Name,Tweet Text,Tweet ID,Bio,Location_clean,preprocessed_tweet,prediction
0,0,Pretoriano,"En un país en manos del crimen organizado, don...",1235653023764013056,"Músico en involución, Ateo consumado, cineasta...",Bogotá,"en un país en manos del crimen organizado , do...",Other
1,1,Yazmín Cortés,"Un hombre tan lindo como tú y con tatuajes, es...",1235653002863706113,Filóloga clásica,Bogotá,"un hombre tan lindo como tú y con tatuajes , e...",Positive
2,2,Javier Duarte,"Otro logro de nuestro subpresidente, ya no nec...",1235652984396304388,Loyalty is the first principle and way to go. ...,Bogotá,"otro logro de nuestro subpresidente , ya no ne...",Other
3,3,𝕷𝖚 🦖.,Detesto todo.,1235652979845365762,muysad.,Bogotá,detesto todo .,Positive
4,4,Santiago Bautista,#YoVotePorDuque yo también voté por @IvanDuque...,1235652968843788288,economista CENTRO DEMOCRATICO,Bogotá,<hashtag> yo vote por duque </hashtag> yo tamb...,Other


In [118]:
dff.drop("Bio", axis = 1, inplace=True)

In [120]:
dft=dff.dropna()

In [121]:
len(dft)

23412

## Prepocessing dataset

In [ ]:
!pip install ekphrasis

In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

In [122]:
dft['preprocessed_tweet'] = dft['Tweet Text'].apply(lambda x: " ".join(text_processor.pre_process_doc(x)))

In [123]:
dft.head()

,Unnamed: 0,Full Name,Tweet Text,Tweet ID,Location_clean,preprocessed_tweet
0,0,Pretoriano,"En un país en manos del crimen organizado, don...",1235653023764013056,Bogotá,"en un país en manos del crimen organizado , do..."
1,1,Yazmín Cortés,"Un hombre tan lindo como tú y con tatuajes, es...",1235653002863706113,Bogotá,"un hombre tan lindo como tú y con tatuajes , e..."
2,2,Javier Duarte,"Otro logro de nuestro subpresidente, ya no nec...",1235652984396304388,Bogotá,"otro logro de nuestro subpresidente , ya no ne..."
3,3,𝕷𝖚 🦖.,Detesto todo.,1235652979845365762,Bogotá,detesto todo .
4,4,Santiago Bautista,#YoVotePorDuque yo también voté por @IvanDuque...,1235652968843788288,Bogotá,<hashtag> yo vote por duque </hashtag> yo tamb...


## Adding prediction

In [124]:
dft['prediction'] = dft['preprocessed_tweet'].apply(lambda x: get_prediction(x)) # pandas df apply

In [126]:
dft.to_csv('datasets/Twitter_1-7_colombia_final_prepro.csv')

In [127]:
grouped_t= dft[['Location_clean','Tweet Text','prediction']].groupby(['Location_clean','prediction'],as_index=False).count()

dep = grouped_t.pivot(index='Location_clean',columns='prediction').fillna(0)

In [128]:
dep.columns=dep.columns.map('{0[0]}{0[1]}'.format) 
dep.reset_index()

,Location_clean,Tweet TextHealthy,Tweet TextNegative,Tweet TextNon-healthy,Tweet TextOther,Tweet TextPositive,Tweet TextPractice_sports
0,Amazonas,14.0,10.0,1.0,194.0,41.0,1.0
1,Antioquia,28.0,76.0,4.0,506.0,132.0,5.0
2,Arauca,12.0,29.0,3.0,284.0,38.0,2.0
3,Atlántico,13.0,52.0,1.0,268.0,100.0,10.0
4,Bogotá,161.0,395.0,184.0,3383.0,642.0,44.0
5,Bolívar,26.0,62.0,6.0,422.0,94.0,11.0
6,Boyacá,15.0,68.0,4.0,361.0,86.0,3.0
7,Caldas,11.0,45.0,1.0,280.0,67.0,5.0
8,Caquetá,8.0,19.0,1.0,156.0,38.0,5.0
9,Casanare,15.0,15.0,2.0,284.0,42.0,4.0


## Model result per departament

In [133]:
dep['Total_tweets']=dep[['Tweet TextHealthy','Tweet TextPractice_sports','Tweet TextPositive','Tweet TextNegative','Tweet TextNon-healthy','Tweet TextOther']].sum(axis=1)
dep['healthy_tweets']=dep[['Tweet TextHealthy','Tweet TextPractice_sports','Tweet TextPositive']].sum(axis=1)
dep['vmd']= dep['healthy_tweets']/dep['Total_tweets']
dep.head()

,Tweet TextHealthy,Tweet TextNegative,Tweet TextNon-healthy,Tweet TextOther,Tweet TextPositive,Tweet TextPractice_sports,Total_tweets,healthy_tweets,vmd
Location_clean,,,,,,,,,
Amazonas,14.0,10.0,1.0,194.0,41.0,1.0,261.0,56.0,0.214559
Antioquia,28.0,76.0,4.0,506.0,132.0,5.0,751.0,165.0,0.219707
Arauca,12.0,29.0,3.0,284.0,38.0,2.0,368.0,52.0,0.141304
Atlántico,13.0,52.0,1.0,268.0,100.0,10.0,444.0,123.0,0.277027
Bogotá,161.0,395.0,184.0,3383.0,642.0,44.0,4809.0,847.0,0.176128


In [135]:
dep[['healthy_tweets','Total_tweets','vmd']].to_csv('datasets/Model_result_departament.csv')